In [ ]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [ ]:
scenarios_df = CSV.read("load_profiles.csv", DataFrame)
n_scenarios = size(scenarios_df, 2)
M = 10000
in_sample_n = 100

In [ ]:
#define function
function f(q)
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)

    @variable(model, cap_bid)
    @variable(model, bin_var[1:60, 1:in_sample_n],Bin)

    @constraint(model, cap_bid >= 0)

    for scenario in 1:in_sample_n
        for minute in 1:60
            @constraint(model, cap_bid - scenarios_df[minute, scenario] <= M* bin_var[minute, scenario])
        end
    end
    @constraint(model, sum(bin_var[minute, scenario] for scenario in 1:in_sample_n, minute in 1:60) <= q)

    # Define the objective function
    @objective(model, Max, cap_bid)

    # Solve the model
    optimize!(model)

    # Check the status of the solution
    status = termination_status(model)
    if status == MOI.OPTIMAL
        #println("Optimal solution found")
        println("Optimal solution cap_bid: ", value(cap_bid))
    elseif status == MOI.INFEASIBLE
        println("No feasible solution found")
    else
        println("Solver status: ", status)
    end
    return JuMP.value.(bin_var)
end

In [ ]:
#ALSO-X

stopping_parameter = 0.0001

lwr_q = 0
upr_q = 0.1*in_sample_n*60
counter = 0

while upr_q - lwr_q >= stopping_parameter
    q = (lwr_q + upr_q)/2
    bin_var = f(q)
    num_zeros = count(x -> x == 0, bin_var)
    println("q: ", q, " num_zeros: ", num_zeros)

    if num_zeros/length(bin_var) > 0.9
        lwr_q = q
    else
        upr_q = q
    end

    if counter > 50
        break
    end
    counter += 1
end